In [1]:
using Revise

using Pkg
Pkg.activate("..")
using FSI

using LinearAlgebra

In [2]:
m = 80
n = 12

# construct a random block system
A = rand(m,m)
A⁻¹ = inv(A)
B₁ᵀ = rand(m,n)
B₂ = rand(n,m)
M = rand(n,n)
M⁻¹ = inv(M)
G₁ᵀ = rand(n,n)
G₂ = rand(n,n)
T₁ᵀ = rand(n,n)
T₂ = rand(n,n)
O32 = zeros(m,n)
O23 = zeros(n,m)
O22 = zeros(n,n)
S = [A B₁ᵀ O32 O32; B₂ O22 -T₂ O22; O23 T₁ᵀ M G₁ᵀ; O23 O22 G₂ O22]

rċ = rand(m)
rf = rand(n)
ru̇ = rand(n)
rλ = rand(n)
b = [rċ;rf;ru̇;rλ];
b1 = [rċ;rf]
b2 = [ru̇;rλ];

In [3]:
# construct saddle point system
ċ = zeros(m)
f = zeros(n)
u̇ = zeros(n)
λ = zeros(n)
St = SaddleSystem1d((ċ, f, u̇, λ), (A⁻¹, B₁ᵀ, B₂),
                  (M, G₁ᵀ, G₂), (x->T₁ᵀ*x, x->T₂*x))

Saddle system with 12 constraints on fluid and 12 constraints on body
   Fluid state of type Array{Float64,1}
   Fluid force of type Array{Float64,1}
   Body state of type Array{Float64,1}
   Joint force of type Array{Float64,1}


In [4]:
@time x_theory = S\b

  0.679373 seconds (2.26 M allocations: 106.819 MiB, 5.12% gc time)


116-element Array{Float64,1}:
  1.386994175210903  
  0.10229598392248983
 -1.0756088684751215 
 -0.28452328006218097
  1.0056187813930964 
  2.812327670994662  
  0.11905432840980469
  0.2529012591369787 
 -1.3225436674507973 
  0.3200376459456192 
  4.0944064679064445 
  2.620833363989784  
  1.8562540484003303 
  ⋮                  
  5.6428826695024155 
 12.876763624260708  
 -2.661925307527669  
 -5.168785533831216  
 -3.4757761614638683 
 -1.3718948472330341 
 14.635316528098306  
 -9.29787447646459   
  6.75263813073742   
 -1.6710539278471617 
 -5.450360155767029  
  5.381926202423524  

In [5]:
bt = (rċ, rf, ru̇, rλ);
aa,bb,cc,dd = St\bt
@time x_compute = [aa;bb;cc;dd]

  0.000005 seconds (6 allocations: 1.266 KiB)


116-element Array{Float64,1}:
  1.3869941752106962 
  0.10229598392258188
 -1.0756088684749894 
 -0.28452328006226946
  1.0056187813930406 
  2.8123276709946525 
  0.11905432840981156
  0.2529012591370312 
 -1.3225436674507487 
  0.3200376459455989 
  4.094406467906365  
  2.620833363989686  
  1.856254048400347  
  ⋮                  
  5.642882669502465  
 12.876763624260636  
 -2.661925307527763  
 -5.168785533830926  
 -3.475776161464023  
 -1.3718948472331924 
 14.635316528098006  
 -9.297874476464528  
  6.752638130737542  
 -1.671053927846936  
 -5.450360155766887  
  5.381926202423403  

In [6]:
norm(x_theory-x_compute)

1.1230414864823329e-12

### check step 1

In [7]:
x1 = [A B₁ᵀ;B₂ O22]\b1
# norm([A B₁ᵀ;B₂ O22]*x1 - [rċ;rf])

92-element Array{Float64,1}:
 -0.6115664225492836
  1.575988639038487 
  0.6001599963062434
 -3.2765771072681837
  0.639757001009569 
  3.4211997251068693
  1.2663142910979843
  1.858901311402029 
 -1.7969927671623067
 -0.5974758417340866
  4.462155732113581 
  0.7015595557128682
  2.337253508609391 
  ⋮                 
 -3.812953747808128 
 -3.193174177786496 
  1.3754421370294192
 -1.3829042857549398
 -1.8997151417171674
 -6.369285331373622 
 -3.962945484590052 
  6.67362878960773  
  8.875152628618672 
 -0.3898338828554046
  2.912675736612643 
 -4.205915608953373 

In [8]:
using ImportMacros
@import ViscousFlow.SaddlePointSystems as Sad

Sf = Sad.SaddleSystem((ċ,f),(A⁻¹,B₁ᵀ,B₂),tol=1e-3,
            issymmetric=false,isposdef=true,store=true,precompile=true)
qq,ww = Sf\(b1[1:m], b1[m+1:end])

([-0.611566, 1.57599, 0.60016, -3.27658, 0.639757, 3.4212, 1.26631, 1.8589, -1.79699, -0.597476  …  2.71226, -3.23387, -1.31032, -4.10057, -0.891322, 0.532656, 1.98995, 2.45495, -2.9921, 5.5905], [-3.81295, -3.19317, 1.37544, -1.3829, -1.89972, -6.36929, -3.96295, 6.67363, 8.87515, -0.389834, 2.91268, -4.20592])

### check step 2

In [9]:
# inv(B₂*A⁻¹*B₁ᵀ)

In [10]:
x_theory[80+13:end]

24-element Array{Float64,1}:
 -0.342460920444509  
  0.13126164891624467
 -0.4313019308988362 
 -0.8287968884384886 
  1.3727079346321347 
  0.635162118683382  
 -0.9279413082890106 
  1.1649879484803243 
 -0.6058628887064851 
 -0.6321817991219384 
  0.04028251373867341
  1.1017259146298235 
  5.6428826695024155 
 12.876763624260708  
 -2.661925307527669  
 -5.168785533831216  
 -3.4757761614638683 
 -1.3718948472330341 
 14.635316528098306  
 -9.29787447646459   
  6.75263813073742   
 -1.6710539278471617 
 -5.450360155767029  
  5.381926202423524  

In [11]:
ee = [M-T₁ᵀ*inv(B₂*A⁻¹*B₁ᵀ)*T₂ G₁ᵀ;G₂ O22]\[b2[1:n]-T₁ᵀ*ww;b2[n+1:end]]

24-element Array{Float64,1}:
 -0.3424609204445004  
  0.131261648916243   
 -0.43130193089884    
 -0.828796888438487   
  1.3727079346321305  
  0.6351621186833893  
 -0.9279413082890151  
  1.1649879484803192  
 -0.605862888706483   
 -0.6321817991219375  
  0.040282513738675904
  1.1017259146298213  
  5.642882669502584   
 12.876763624258293   
 -2.661925307528189   
 -5.168785533830195   
 -3.4757761614638567  
 -1.3718948472328607  
 14.635316528095636   
 -9.297874476463813   
  6.752638130736341   
 -1.6710539278461212  
 -5.450360155765503   
  5.381926202422637   

In [12]:
using IterativeSolvers
bicgstabl([M-T₁ᵀ*inv(B₂*A⁻¹*B₁ᵀ)*T₂ G₁ᵀ;G₂ O22], [b2[1:n]-T₁ᵀ*ww;b2[n+1:end]], tol=1e-3)

24-element Array{Float64,1}:
  0.6563179973194134  
  0.27735726072418476 
 -0.22388963531565542 
 -0.3532232034997813  
  0.9018939926098217  
 -0.9379213652237981  
 -0.47996323754816506 
 -1.2266348687850217  
  0.3360417803723163  
  0.22893135987095026 
  1.3444608576175903  
 -0.6454096385298547  
 -0.22089672318071518 
  1.0564893224892513  
 -0.8260134246674561  
 -0.12244136908117143 
  0.14193387832403537 
  1.7420509647444802  
  1.4136560371735953  
 -0.33831107105478453 
  0.010233364784144161
 -0.6534745524929988  
 -0.27027293010764475 
  0.6072060433306814  

### check step 3

In [13]:
ċ = qq + A⁻¹*B₁ᵀ*inv(B₂*A⁻¹*B₁ᵀ)*T₂*ee[1:n]

80-element Array{Float64,1}:
  1.3869941752108608 
  0.10229598392243444
 -1.0756088684751608 
 -0.28452328006164196
  1.0056187813929571 
  2.8123276709942298 
  0.11905432840963659
  0.2529012591368549 
 -1.3225436674506839 
  0.3200376459456765 
  4.094406467905998  
  2.620833363989786  
  1.8562540484000787 
  ⋮                  
 -1.9829662481836388 
  0.1442239195034487 
 -1.913454987029347  
 -3.113458244362346  
  1.9566590716515813 
 -1.9699211229663542 
  0.36450478217346527
 -1.1140745297534926 
  1.011279745311979  
  2.8423307311343975 
 -2.2519205057017997 
  4.598979795058751  

In [14]:
f = f - inv(B₂*A⁻¹*B₁ᵀ)*T₂*ee[1:n]

12-element Array{Float64,1}:
  4.125963547276349  
  0.5518370739885867 
 -0.5395264527005404 
 -0.19531428807845275
  1.230430960445866  
  0.6597641302973376 
  5.122567465641303  
 -1.778055513780092  
 -3.8054370690551655 
  0.5027324132065203 
 -3.7331598960083827 
  0.9170561297889712 